In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [3]:
from typing import List, Tuple, Dict
import math
from models.cnn import CNN
from data_loader.cifar10 import DataLoader
from utils import helper

In [ ]:
myint = tf.int32
myfloat = tf.float32

In [ ]:
datasource = DataLoader()

In [ ]:
print(datasource.data.shape)
print(datasource.labels.shape)
print(datasource.test_labels.shape)
print(datasource.num_step(batch_size))

# network

In [ ]:
cnn = CNN()

In [ ]:
crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=cnn.y, logits=cnn.logits)
loss_op = tf.reduce_mean(crossent)
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)
correct = tf.equal(cnn.predicted_classes, cnn.y)
acc_op = tf.reduce_mean(tf.cast(correct, myfloat))

In [ ]:
with tf.name_scope('train'):
    smr_loss = tf.summary.scalar('loss', loss_op)
    smr_acc = tf.summary.scalar('accuracy', acc_op)
    merged_summary = tf.summary.merge([smr_loss, smr_acc])

with tf.name_scope('test'):
    test_smr_acc = tf.summary.scalar('accuracy', acc_op)

In [ ]:
from datetime import datetime
now = datetime.now()
logdir_base = 'logs/'

In [ ]:
logdir = logdir_base + now.strftime("%Y%m%d-%H%M%S") + "/"
with tf.Session() as sess:
    writer = tf.summary.FileWriter(logdir, sess.graph)
    sess.run(tf.global_variables_initializer())
    
    global_step = 0
    for i in range(num_epoch):
        step_size = datasource.num_step(batch_size)
        for s in range(step_size):
            data, labels = datasource.next_batch(batch_size)
            fd = {
                cnn.x: data,
                cnn.y: labels,
                cnn.is_training: True
            }
            loss, _, acc, smr = sess.run([loss_op, train_op, acc_op, merged_summary], feed_dict=fd)
            if s % num_print_step == 0:
                writer.add_summary(smr, global_step)
                print('{} steps, train accuracy: {:.6f}, loss: {:.6f}'.format(global_step, acc, loss))
                test_acc, test_smr = sess.run([acc_op, test_smr_acc], feed_dict={
                    cnn.x: datasource.test_data,
                    cnn.y: datasource.test_labels,
                    cnn.is_training: False
                })
                writer.add_summary(test_smr, global_step)
            global_step += 1
        print('{} steps, test accuracy:  {:.6f}, loss: {:.6f} ({}/{} epochs)'.format(global_step, test_acc, loss, i, num_epoch))